# Demo of unified error report via TVMScript printer

Error report is always an important part of UX for both developers and users. For example, 

1. `assert_structural_equal` is widely used in the TVM codebase, especially in the unit tests. It not only checks the equality of two IR structures, but also provide critical error information when inequality exists. Those error reported are useful for developer to debug further. 

2. Our `ScheduleError::RenderReport` report renders the TVMScript with annotations and underlines, to elaborate the errors occurred during the schedule. So users may quickly locate the errors and fix them soon.

However, in the past, these error report components are designed and work independently:

1. `assert_structural_equal` directly prints the mismatching IR nodes without any context. For example, it may report

    > ValueError: StructuralEqual check failed, caused by lhs:
    > 128
    > and rhs:
    > 256

    and it is hard to locate which fields are mismatched with 128 and 256 in the whole script.

2. `ScheduleError::RenderReport` has its own printer as `AsTVMScriptWithDiagnostic` to support annotations and underlines rendering.

Fortunately, these error report componnents have been unified with the TVMScript printer. Since this TVMScript printer is designed naturally for debugging, annotating and underlining beyond the basic printing.

## Underline

Let us start with the following scripts:

In [1]:
import tvm
from tvm.ir import assert_structural_equal
from tvm.script import tir as T


@T.prim_func
def func1(a: T.handle, b: T.handle):
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 128))  # shape mismatched

@T.prim_func
def func2(a: T.handle, b: T.handle):
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 256)) # shape mismatched

The two scripts look almost the same, except the buffer shapes of `B`. In `func1`, the buffer `B` is of shape `(128, 128)`. But in `func2`, the shape of buffer `B` is `(128, 256)`. Then let us run `assert_structural_equal` for them, and skip the trace back stack.

In [2]:
try:
    assert_structural_equal(func1, func2, True)
except ValueError as ve:
    print(f"ValueError{str(ve).split('ValueError')[1]}")

ValueError: StructuralEqual check failed, caused by lhs at <root>.buffer_map[b].shape[1].value:
# from tvm.script import tir as T

@T.prim_func
def main(a: T.handle, b: T.handle):
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 128))
                                ^^^
    T.evaluate(0)
and rhs at <root>.buffer_map[b].shape[1].value:
# from tvm.script import tir as T

@T.prim_func
def main(a: T.handle, b: T.handle):
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 256))
                                ^^^
    T.evaluate(0)


The error reported includes the two parts:
1. The `ObjectPath` to the mismatching nodes.
2. The TVMScript with underlines mapping the mismatching nodes.
So it is convenient to access the mismatching nodes as below:

In [3]:
lhs_obj = func1.buffer_map[func1.params[1]].shape[1]
rhs_obj = func2.buffer_map[func2.params[1]].shape[1]
print(f"The value at func1.buffer_map[b].shape[1].value is: {lhs_obj.value}")
print(f"The value at func2.buffer_map[b].shape[1].value is: {rhs_obj.value}")

The value at func1.buffer_map[b].shape[1].value is: 128
The value at func2.buffer_map[b].shape[1].value is: 256


The underlined TVMScript rendering is similar to below logic:

In [4]:
from tvm.ir.base import get_first_structural_mismatch

lhs_obj_path, rhs_obj_path = get_first_structural_mismatch(func1, func2)
print("The underlined func1:")
func1.show(black_format=False, syntax_sugar=False, path_to_underline=[lhs_obj_path])
print("The underlined func2:")
func2.show(black_format=False, syntax_sugar=False, path_to_underline=[rhs_obj_path])

The underlined func1:


The underlined func2:


Here, we set `syntax_sugar=False` to show complete script and use `path_to_underline` parameter to underline our script based on `ObjectPath`. Similarly, TVMScript printer provides another `ObjectRef`-based interface, `obj_to_underline`, to achieve same result:

In [5]:
assert func1.script(syntax_sugar=False, 
                    obj_to_underline=[lhs_obj]) \
    == func1.script(syntax_sugar=False, 
                    path_to_underline=[lhs_obj_path])

But `obj_to_underline` is able to underline all usages of the `ObjectRef`, while `path_to_underline` focuses on certain node or location: 

In [6]:
@T.prim_func
def func3(a: T.int32, b: T.int32):
  T.evaluate(a)
  T.evaluate(b)
  T.evaluate(a)
  T.evaluate(b)

func3.show(black_format=False, obj_to_underline=[func3.params[0]])

## Annotation

Besides of underlining, the TVMScript printer supports annotation naturally for error reporting.

- `obj_to_annotate`
- `path_to_annotate`

Note: the `*_to_annotate` methods annoate statement nodes only, i.e. `FunctionDef`, `For`, `Let`, etc. But not for expression nodes.

Here is an example usage as `ScheduleError::RenderReport`:

In [7]:
from tvm import tir

@T.prim_func
def func4(a: T.handle, b: T.handle) -> None:
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 128))
    for i, j in T.grid(128, 128):
        with T.block("block"):
            vi, vj = T.axis.remap("SS", [i, j])
            B[vi, vj] = A[vi, vj] * 2.0

func4.show(black_format=False,
           syntax_sugar=False,
           obj_to_annotate={func4.body: "Block#0",
                            func4.body.block.body: "For#1",
                            func4.body.block.body.body: "For#2",
                            func4.body.block.body.body.body: "Block#3"})